（10月5日）**任务1**：报名比赛，下载比赛数据集并完成读取

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/tianchi-nlp-intro

/content/drive/MyDrive/tianchi-nlp-intro


In [3]:
!pwd

/content/drive/MyDrive/tianchi-nlp-intro


In [4]:
!nvidia-smi

Sat Oct  9 03:44:02 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import pandas as pd
train_df = pd.read_csv('./train_set.csv', sep='\t', nrows=100)
train_df['word'] = train_df['text'].apply(lambda x: len(x.split(' ')))

（10月5日）**任务2**：对数据集字符进行可视化，统计标签和字符分布
- [x] 统计数据集中所有句子所包含字符的平均个数
- [x] 统计数据集中不同类别下句子平均字符的个数
- [x] 统计数据集中类别分布的规律
- [x] 统计数据集中不同类别下句子中最常见的5个字符

In [ ]:
label2id = {'科技': 0, '股票': 1, '体育': 2, '娱乐': 3, '时政': 4, '社会': 5, '教育': 6, '财经': 7, '家居': 8, '游戏': 9, '房产': 10, '时尚': 11, '彩票': 12, '星座': 13}
id2label = {v:k for k, v in label2id.items()}

In [ ]:
avg = train_df['word'].mean()
print(f'统计数据集中所有句子所包含字符的平均个数: {avg:.2f}')

统计数据集中所有句子所包含字符的平均个数: 872.32


In [ ]:
avg_by_label = train_df.groupby('label')['word'].mean()
for i, avg in enumerate(avg_by_label):
    label = id2label[i]
    print(f'\"{label}\"类别下的字符平均个数：{avg:.2f}')

"科技"类别下的字符平均个数：1096.54
"股票"类别下的字符平均个数：721.56
"体育"类别下的字符平均个数：1104.63
"娱乐"类别下的字符平均个数：512.17
"时政"类别下的字符平均个数：329.12
"社会"类别下的字符平均个数：1516.67
"教育"类别下的字符平均个数：2214.20
"财经"类别下的字符平均个数：1012.00
"家居"类别下的字符平均个数：435.67
"游戏"类别下的字符平均个数：565.67
"房产"类别下的字符平均个数：1177.67
"时尚"类别下的字符平均个数：425.00
"彩票"类别下的字符平均个数：374.00
"星座"类别下的字符平均个数：131.00


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

label_counts = train_df['label'].value_counts()
labels = [label for label in label2id]
plt.bar(labels, label_counts)
plt.title('数据集中类别分布情况')
plt.xlabel('类别')
plt.ylabel('数量')
plt.show()

In [ ]:
print('统计数据集中不同类别下句子中最常见的5个字符:\n')
import collections

for i in range(len(labels)):
    df = train_df[train_df['label'] == i]
    words_list = []
    for text in df['text']:
        words_list += text.split()
    words_counter = collections.Counter(words_list)
    print(i, id2label[i], words_counter.most_common(5))

统计数据集中不同类别下句子中最常见的5个字符:

0 科技 [('3750', 610), ('648', 458), ('900', 298), ('6122', 179), ('3370', 150)]
1 股票 [('3750', 531), ('648', 314), ('3370', 211), ('4464', 189), ('900', 186)]
2 体育 [('3750', 956), ('648', 541), ('900', 434), ('4704', 225), ('1667', 212)]
3 娱乐 [('3750', 239), ('648', 127), ('900', 87), ('2465', 65), ('4939', 62)]
4 时政 [('3750', 78), ('648', 63), ('900', 56), ('4464', 40), ('4411', 35)]
5 社会 [('3750', 193), ('648', 135), ('900', 98), ('4939', 66), ('5598', 53)]
6 教育 [('3750', 491), ('648', 450), ('2555', 297), ('5620', 215), ('900', 207)]
7 财经 [('3750', 214), ('3370', 134), ('648', 106), ('900', 100), ('4464', 85)]
8 家居 [('3750', 68), ('648', 42), ('913', 33), ('5305', 21), ('4326', 20)]
9 游戏 [('3750', 51), ('3370', 35), ('3317', 31), ('7010', 27), ('7539', 26)]
10 房产 [('3750', 152), ('648', 84), ('2999', 64), ('900', 57), ('408', 57)]
11 时尚 [('3750', 102), ('648', 73), ('900', 40), ('5310', 27), ('3809', 24)]
12 彩票 [('4464', 59), ('6220', 59), ('3659', 46), ('337

**（10月11日）任务3：使用 TFIDF 提取文本特征**

- [x] 学习TFIDF的原理
- [x] 学会使用CountVectorizer
- [x] 学会使用TfidfVectorizer

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

train_df = pd.read_csv('./train_set.csv', sep='\t')
test_df = pd.read_csv('./test_a.csv', sep='\t')

In [ ]:
train_df.head()

,label,text
0,2,2967 6758 339 2021 1854 3731 4109 3792 4149 15...
1,11,4464 486 6352 5619 2465 4802 1452 3137 5778 54...
2,3,7346 4068 5074 3747 5681 6093 1777 2226 7354 6...
3,2,7159 948 4866 2109 5520 2490 211 3956 5520 549...
4,3,3646 3055 3055 2490 4659 6065 3370 5814 2465 5...


In [ ]:
X_train = train_df['text']
y_train = train_df['label']

count_vec = CountVectorizer()
count_X_train = count_vec.fit_transform(X_train)

tfidf_vec = TfidfVectorizer()
tfidf_X_train = tfidf_vec.fit_transform(X_train)

**（10月11日）任务4：使用 TFIDF 特征 和 线性模型完成训练和预测**

- [ ] 使用TFIDF提取训练集和测试集特征
- [ ] 使用线性模型（LR等）完成模型的训练和预测

In [ ]:
import pandas as pd
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer

train_df = pd.read_csv('./train_set.csv', sep='\t')
test_df = pd.read_csv('./test_a.csv', sep='\t')

X_train = train_df['text']
X_test = test_df['text']
X_train_and_test = pd.concat([X_train, X_test])

tfidf = TfidfVectorizer()
tfidf.fit(X_train_and_test)
tfidf_X_train = tfidf.transform(X_train)
tfidf_X_test = tfidf.transform(X_test)



In [ ]:
from scipy import sparse
import numpy as np
import pickle

# 保存模型及数据
pickle.dump(tfidf, open('./model/tfidf.model', 'wb'))
sparse.save_npz('./data/tfidf_X_train.npz', tfidf_X_train)
sparse.save_npz('./data/tfidf_X_test.npz', tfidf_X_test)
np.save('./data/y_train.npy', train_df['label'])


In [ ]:
from scipy import sparse
import numpy as np

# 加载数据
X_train = sparse.load_npz('./data/tfidf_X_train.npz')
y_train = np.load('./data/y_train.npy')
X_test = sparse.load_npz('./data/tfidf_X_test.npz')



In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

fold = StratifiedKFold(n_splits=5, shuffle=True)
n_class = 14

y_pred_prob = np.zeros((X_test.shape[0], n_class), dtype=np.float32)
lr = LogisticRegression()

for i, (train_idx, valid_idx) in enumerate(fold.split(X_train, y_train)):
    print('第', i+1, '折交叉验证开始...')
    X_train_, X_valid_ = X_train[train_idx], X_train[valid_idx]
    y_train_, y_valid_ = y_train[train_idx], y_train[valid_idx]
    lr.fit(X_train_, y_train_)
    valid_preds = lr.predict(X_valid_)
    valid_score = f1_score(y_valid_, valid_preds, average='macro')
    print(f'f1 score: {valid_score:.4f}')
    y_pred_prob += lr.predict_proba(X_test)
    pickle.dump(lr, open(f'./model/lr_{i}.model', 'wb'))

In [ ]:
preds = y_pred_prob.argmax(axis=-1)
submit_lr = pd.DataFrame(preds, columns=['label'])
submit_lr.to_csv('./submit/submit_lr_2.csv', index=False)

**（10月11日）任务5：使用 TFIDF 特征 和 XGBoost完成训练和预测**

- [ ] 使用TFIDF提取训练集和测试集特征
- [ ] 使用XGBoost完成模型的训练和预测

In [5]:
from scipy import sparse
import numpy as np

# 加载数据
X_train = sparse.load_npz('./data/tfidf_X_train.npz')
y_train = np.load('./data/y_train.npy')
X_test = sparse.load_npz('./data/tfidf_X_test.npz')

In [7]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=682)
svd.fit(X_train)
svd_X_train = svd.transform(X_train)
svd_X_test = svd.transform(X_test)

X_train_, X_valid_, y_train_, y_valid_ = train_test_split(svd_X_train, y_train, test_size=0.3, random_state=0)


xg_params = {
    'tree_method': 'gpu_hist',
    'booster': 'gbtree',
    'objective': 'multi:softprob',
    'num_class': 14,
    'max_depth': 10,
    'eta': 0.05,
    'subsample': 0.8,
    'nthread': -1,
    'seed':0,
}

def xgb_f1(y_pred: np.ndarray, dtrain: xgb.DMatrix):
    ''' F1 score metric''' 
    y_true = dtrain.get_label()
    y_pred = y_pred.argmax(axis=-1)
    return 'f1_score', f1_score(y_true, y_pred, average='macro')


xg_train = xgb.DMatrix(X_train_, label=y_train_)
xg_valid = xgb.DMatrix(X_valid_, label=y_valid_)
xg_test = xgb.DMatrix(svd_X_test)

num_rounds = 200
watchlist = [(xg_train, 'train'), (xg_valid, 'val')] 

xg_model = xgb.train(params=xg_params, 
                     dtrain=xg_train, 
                     num_boost_round=num_rounds, 
                     evals=watchlist, 
                     feval=xgb_f1)

xg_model.save_model('./model/xgb.model')


[0]	train-merror:0.115364	val-merror:0.171767	train-f1_score:0.858263	val-f1_score:0.782164
[1]	train-merror:0.098014	val-merror:0.1512	train-f1_score:0.880306	val-f1_score:0.808894
[2]	train-merror:0.090157	val-merror:0.143733	train-f1_score:0.890404	val-f1_score:0.818511
[3]	train-merror:0.085257	val-merror:0.138733	train-f1_score:0.89798	val-f1_score:0.826227
[4]	train-merror:0.082707	val-merror:0.136417	train-f1_score:0.901291	val-f1_score:0.829267
[5]	train-merror:0.080171	val-merror:0.1342	train-f1_score:0.904685	val-f1_score:0.832508
[6]	train-merror:0.0785	val-merror:0.1329	train-f1_score:0.907249	val-f1_score:0.835165
[7]	train-merror:0.076529	val-merror:0.132233	train-f1_score:0.910077	val-f1_score:0.835981
[8]	train-merror:0.075264	val-merror:0.13105	train-f1_score:0.911408	val-f1_score:0.837784
[9]	train-merror:0.073693	val-merror:0.129883	train-f1_score:0.91361	val-f1_score:0.839439
[10]	train-merror:0.072064	val-merror:0.129283	train-f1_score:0.915576	val-f1_score:0.83974

In [9]:
y_pred_prob = xg_model.predict(xg_test)
y_pred = y_pred_prob.argmax(axis=-1)
submit_xgb = pd.DataFrame(y_pred, columns=['label'])
submit_xgb.to_csv('./submit/submit_xgb.csv', index=False)

In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=682)
svd.fit(X_train)
svd_X_train = svd.transform(X_train)
svd_X_test = svd.transform(X_test)

X_train_, X_valid_, y_train_, y_valid_ = train_test_split(svd_X_train, y_train, test_size=0.3, random_state=0)


xg_params = {
    'tree_method': 'gpu_hist',
    'booster': 'gbtree',
    'objective': 'multi:softprob',
    'num_class': 14,
    'max_depth': 6,
    'eta': 0.01,
    'subsample': 0.8,
    'colsample_bytree': 0.8
    'nthread': -1,
    'seed':0,
}

def xgb_f1(y_pred: np.ndarray, dtrain: xgb.DMatrix):
    ''' F1 score metric''' 
    y_true = dtrain.get_label()
    y_pred = y_pred.argmax(axis=-1)
    return 'f1_score', f1_score(y_true, y_pred, average='macro')


xg_train = xgb.DMatrix(X_train_, label=y_train_)
xg_valid = xgb.DMatrix(X_valid_, label=y_valid_)
xg_test = xgb.DMatrix(svd_X_test)

num_rounds = 200
watchlist = [(xg_train, 'train'), (xg_valid, 'val')] 

xg_model = xgb.train(params=xg_params, 
                     dtrain=xg_train, 
                     num_boost_round=num_rounds, 
                     evals=watchlist, 
                     feval=xgb_f1)

xg_model.save_model('./model/xgb.model')


In [13]:
xg_params = {
    'tree_method': 'gpu_hist',
    'booster': 'gbtree',
    'objective': 'multi:softprob',
    'num_class': 14,
    'max_depth': 6,
    'eta': 0.1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'nthread': -1,
    'seed':0
}

def xgb_f1(y_pred: np.ndarray, dtrain: xgb.DMatrix):
    ''' F1 score metric''' 
    y_true = dtrain.get_label()
    y_pred = y_pred.argmax(axis=-1)
    return 'f1_score', f1_score(y_true, y_pred, average='macro')


xg_train = xgb.DMatrix(X_train_, label=y_train_)
xg_valid = xgb.DMatrix(X_valid_, label=y_valid_)
xg_test = xgb.DMatrix(svd_X_test)

num_rounds = 200
watchlist = [(xg_train, 'train'), (xg_valid, 'val')] 

xg_model = xgb.train(params=xg_params, 
                     dtrain=xg_train, 
                     num_boost_round=num_rounds, 
                     evals=watchlist, 
                     feval=xgb_f1)


[0]	train-merror:0.194993	val-merror:0.209933	train-f1_score:0.755721	val-f1_score:0.728319
[1]	train-merror:0.165671	val-merror:0.1791	train-f1_score:0.794589	val-f1_score:0.770221
[2]	train-merror:0.158593	val-merror:0.171483	train-f1_score:0.805595	val-f1_score:0.781856
[3]	train-merror:0.154529	val-merror:0.16715	train-f1_score:0.812313	val-f1_score:0.788848
[4]	train-merror:0.151879	val-merror:0.16455	train-f1_score:0.816508	val-f1_score:0.793591
[5]	train-merror:0.151429	val-merror:0.163833	train-f1_score:0.815863	val-f1_score:0.793634
[6]	train-merror:0.149629	val-merror:0.162367	train-f1_score:0.818158	val-f1_score:0.795474
[7]	train-merror:0.148621	val-merror:0.162267	train-f1_score:0.818676	val-f1_score:0.794097
[8]	train-merror:0.147107	val-merror:0.159683	train-f1_score:0.820386	val-f1_score:0.797281
[9]	train-merror:0.144771	val-merror:0.15845	train-f1_score:0.822614	val-f1_score:0.797928
[10]	train-merror:0.144	val-merror:0.15765	train-f1_score:0.823102	val-f1_score:0.799